## 협업 필터링 (Collaborative Filtering) 구현하기 

이번 수업에서는 추천 시스템(Recommender System)에서 널리 사용되는 협업 필터링(이하 Collaborative Filtering)의 원리를 알아보고 이를 구현해보겠습니다. 추천 시스템은 사용자(이하 사용자)가 특정 물건이나 서비스(이하 상품)에 대한 선호 여부나 선호도를 예측하는 시스템을 의미합니다. 추천 시스템은 아마존과 같은 이커머스부터 페이스북과 같은 SNS, 유튜브, 넷플릭스 등과 같은 동영상 플랫폼까지 다양한 분야에서 두루 활용되고 있습니다.

Collaborative Filtering에는 사용자에게 상품을 추천을 방법이 크게 두 가지가 있습니다. 1. 사용자가 선호하는 상품과 유사한 다른 상품 을 추천(상품 기반)하거나 2. 사용자와 유사한 다른 사용자가 선호하는 상품을 추천(사용자 기반)합니다. 사용자 기반 기법이 먼저 등장한 전통적인 알고리즘이고 상품 기반 방식은 이후 아마존(Amazon)이 제안한 기법입니다. 상품 기반 기법이 더 많은 기업들에서 사용되고 있다고 합니다.

사용자 기반 방식이 갖는 문제는 우선 **1. 계산 복잡성 문제**와 **2. 희소성 문제**가 대표적입니다. 아마존과 같이 거대 이커머스 회사들은 수백만 명의 사용자와 수백만 개의 상품을 관리해야하는데 사용자 기반 방식을 사용하는 경우 사용자가 추가될 때마다 나머지 모든 사용자와의 유사도를 연산해야한다는 문제점이 있습니다. 상품 기반 방식을 사용하는 경우에 미리 구해 놓은 상품 간 유사도를 활용할 수 있기 때문에 이러한 문제점이 어느 정도 해결됩니다! 물론 상품 기반 방식도 상품과 사용자가 계속 추가되므로 일정 기간마다 새롭게 유사도를 구해야하지만 사용자 기반 방식보다는 훨씬 계산 복잡성이 작습니다. 그리고 계산 복잡성 문제가 해결되는 대신 이 거대한 행렬을 저장할 공간이 따로 확보되어야한다는 점을 굳이 단점으로 뽑을 수 있습니다. 데이터 희소성 문제는 협업 필터링 알고리즘의 본질적인 취약한 점이지만 사용자가 많은 상품을 평가한 경우는 보통 없어서 이런 경우 사용자간의 유사도를 연산하는 것 자체가 어렵기 때문에 보통 사용자 기반 방식이 더 취약합니다.


상품 / 사용자 기반 기법은 전반적으로 다음과 같은 흐름으로 동작합니다.

1. 우선 사용자 $u$가 내릴 상품 $i$에 대한 평점(rating)을 추정하고자 합니다. 상품 $i$ / 사용자 $u$와 나머지 모든 상품 / 사용자의 유사도를 연산합니다.
2. 유사도가 높은 k개 상품 / 사용자를 선택합니다. 이를 이웃이라고 부르겠습니다.
3. 상품 기반 혹은 사용자 기반 기법에 따라 아래 단계를 수행하며 평점을 예측합니다.
    - 상품 기반 : 이웃 상품에 내린 사용자 $u$의 평점(rating)을 상품 $i$와의 유사도에 따라 가중 평균을 구합니다. 
    - 사용자 기반 : 이웃 사용자가 상품 $i$에 내린 평점(rating)을 사용자 $u$와의 유사도에 따라 가중 평균을 구합니다.
4. 아직 평점(rating)이 없는 항목에 대해 모든 평점(rating)을 예측합니다. 평점(rating) 예측 값 상위 n개 상품을 추천합니다.

이러한 머신러닝 알고리즘을 잘 이해하는 방법은, 알고리즘을 파이썬과 같은 프로그래밍 언어로 직접 구현해보는 것입니다. 그러므로 이번 시간에는 주어진 데이터와 문제를 Collaborative Filtering을 활용하여 풀되, [surprise](http://surpriselib.com/)와 같은 추천 시스템 패키지를 사용하지 않고 파이썬으로 직접 구현해서 풀어보는 시간을 가질 것입니다.

### Configuration

In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame

### Data Loader

In [3]:
data = pd.read_csv("../data/ratings.csv")
data

,사람,책,평점
0,민지,백설공주,5.0
1,민지,신데렐라,4.0
2,민지,어린왕자,1.0
3,민지,흥부전,3.0
4,현우,노인과바다,3.0
5,현우,신데렐라,2.0
6,현우,콩쥐팥쥐,1.0
7,현우,흥부전,2.0
8,민수,노인과바다,3.0
9,민수,백설공주,4.0


In [3]:
print(data.shape)

(23, 3)


In [4]:
ratings = data.pivot_table(index="사람",values="평점",columns="책")
ratings

책,노인과바다,백설공주,신데렐라,어린왕자,콩쥐팥쥐,흥부전
사람,,,,,,
민수,3.0,4.0,4.0,3.0,4.0,NaN
민지,NaN,5.0,4.0,1.0,NaN,3.0
지민,4.0,1.0,NaN,5.0,2.0,3.0
지연,5.0,NaN,3.0,4.0,3.0,3.0
현우,3.0,NaN,2.0,NaN,1.0,2.0


In [4]:
ratings = data.pivot_table(index="사람",values="평점",columns = "책") # 사용자 중심
#ratings = data.pivot_table(index="책",values="평점",columns = "사람") # 아이템 중심
ratings

책,노인과바다,백설공주,신데렐라,어린왕자,콩쥐팥쥐,흥부전
사람,,,,,,
민수,3.0,4.0,4.0,3.0,4.0,NaN
민지,NaN,5.0,4.0,1.0,NaN,3.0
지민,4.0,1.0,NaN,5.0,2.0,3.0
지연,5.0,NaN,3.0,4.0,3.0,3.0
현우,3.0,NaN,2.0,NaN,1.0,2.0


In [21]:
'''
유사도를 계산할 때
평점이 0인 것과 NaN인 것은 상당히 큰 차이가 있다.
0은 유사성 값이 존재하는 반면에, NaN은 유사성 값이 존재하지 않는 것이기 때문에
NaN값에 해당하는 컬럼을 없애고 나서 유사도를 계산해야 한다.

결론은 NaN값을 가지고 있는 컬럼은 
유사도를 계산하는 데이터로 판단될 수 없다. 제외시킨다.
'''

u = np.array([np.nan,4,3]) #u라는 사용자의 제품에 대한 평점
v = np.array([np.nan,2,4])
u
v

mask = np.isfinite(u) & np.isfinite(v) #nan이 아닌 값만 True로 표시
mask

u = u[mask]
v = v[mask]
print(u)
print(v)

[4. 3.]
[2. 4.]


### mask 씌우기
    평점이 입력되지 않은 컬럼은 유사도 계산에서 제껴버리자

In [22]:
def get_cosine_similarity(u,v):
    mask = np.isfinite(u) & np.isfinite(v)
    u = u[mask]
    v = v[mask]

    print(u)
    print(v)

    # 코사인 유사도로 계산

    # 분자
    uvdot = (u * v).sum() 

    # 분모
    norm1 = (u**2).sum()
    norm2 = (v**2).sum()

    score = uvdot / np.sqrt(norm1 * norm2)

    return score

In [23]:
get_cosine_similarity(u,v)

[4. 3.]
[2. 4.]


0.8944271909999159

In [25]:
## 민수, 민지에 대한 가로행 정보를 받아온다.
u = ratings.loc["민수"]
v = ratings.loc["민지"]
print("*"*30)
print(u)
print("*"*30)
print(v)

get_cosine_similarity(u,v)

******************************
책
노인과바다    3.0
백설공주     4.0
신데렐라     4.0
어린왕자     3.0
콩쥐팥쥐     4.0
흥부전      NaN
Name: 민수, dtype: float64
******************************
책
노인과바다    NaN
백설공주     5.0
신데렐라     4.0
어린왕자     1.0
콩쥐팥쥐     NaN
흥부전      3.0
Name: 민지, dtype: float64
책
백설공주    4.0
신데렐라    4.0
어린왕자    3.0
Name: 민수, dtype: float64
책
백설공주    5.0
신데렐라    4.0
어린왕자    1.0
Name: 민지, dtype: float64


0.9398272507881658

In [26]:
ratings

책,노인과바다,백설공주,신데렐라,어린왕자,콩쥐팥쥐,흥부전
사람,,,,,,
민수,3.0,4.0,4.0,3.0,4.0,NaN
민지,NaN,5.0,4.0,1.0,NaN,3.0
지민,4.0,1.0,NaN,5.0,2.0,3.0
지연,5.0,NaN,3.0,4.0,3.0,3.0
현우,3.0,NaN,2.0,NaN,1.0,2.0


In [27]:
ratings.index

Index(['민수', '민지', '지민', '지연', '현우'], dtype='object', name='사람')

### 모든 사람의 유사도를 검색
    파이썬에서는 조합, 순열을 만들 때 사용하는 유용한 모듈이 있다.
    itertools 모듈을 이럴 때 사용한다.
    지금의 경우는 조합을 알아보는 계산
    민수-민지, 민수-민수, 민수-지민, 민수-지현...

In [5]:
from itertools import product

In [40]:
def get_cosine_similarity_table(ratings):
    # ratings.index에서 2명씩 묶겠다....
    index_combinations = list(product(ratings.index,repeat=2))
    similarity_list = []

    for uname, vname in index_combinations:
        u, v = ratings.loc[uname], ratings.loc[vname]
        score = get_cosine_similarity(u,v)
        similarity = { # json 형식
            "u" : uname,
            "v" : vname,
            "score" : score
        }

        similarity_list.append(similarity)


    similarity_list = pd.DataFrame(similarity_list)

    similarity_table = pd.pivot_table(similarity_list, index="u",columns="v",values="score")
    return similarity_table

In [42]:
similarity_table=get_cosine_similarity_table(ratings)
similarity_table

책
노인과바다    3.0
백설공주     4.0
신데렐라     4.0
어린왕자     3.0
콩쥐팥쥐     4.0
Name: 민수, dtype: float64
책
노인과바다    3.0
백설공주     4.0
신데렐라     4.0
어린왕자     3.0
콩쥐팥쥐     4.0
Name: 민수, dtype: float64
책
백설공주    4.0
신데렐라    4.0
어린왕자    3.0
Name: 민수, dtype: float64
책
백설공주    5.0
신데렐라    4.0
어린왕자    1.0
Name: 민지, dtype: float64
책
노인과바다    3.0
백설공주     4.0
어린왕자     3.0
콩쥐팥쥐     4.0
Name: 민수, dtype: float64
책
노인과바다    4.0
백설공주     1.0
어린왕자     5.0
콩쥐팥쥐     2.0
Name: 지민, dtype: float64
책
노인과바다    3.0
신데렐라     4.0
어린왕자     3.0
콩쥐팥쥐     4.0
Name: 민수, dtype: float64
책
노인과바다    5.0
신데렐라     3.0
어린왕자     4.0
콩쥐팥쥐     3.0
Name: 지연, dtype: float64
책
노인과바다    3.0
신데렐라     4.0
콩쥐팥쥐     4.0
Name: 민수, dtype: float64
책
노인과바다    3.0
신데렐라     2.0
콩쥐팥쥐     1.0
Name: 현우, dtype: float64
책
백설공주    5.0
신데렐라    4.0
어린왕자    1.0
Name: 민지, dtype: float64
책
백설공주    4.0
신데렐라    4.0
어린왕자    3.0
Name: 민수, dtype: float64
책
백설공주    5.0
신데렐라    4.0
어린왕자    1.0
흥부전     3.0
Name: 민지, dtype: float64
책
백설공주    5.0
신데렐라    4.0
어린왕자    1.0
흥부전

v,민수,민지,지민,지연,현우
u,,,,,
민수,1.000000,0.939827,0.813206,0.938986,0.876523
민지,0.939827,1.000000,0.542857,0.840841,0.989949
지민,0.813206,0.542857,1.000000,0.974406,0.992583
지연,0.938986,0.840841,0.974406,1.000000,0.980581
현우,0.876523,0.989949,0.992583,0.980581,1.000000


### 평점 예측하기

In [44]:
ratings["노인과바다"]

사람
민수    3.0
민지    NaN
지민    4.0
지연    5.0
현우    3.0
Name: 노인과바다, dtype: float64

In [54]:
'''
모든 사람들의 유사도를 파악한 결과를 바탕으로
민지의 노인과바다 책에 관한 평점을 예측해보자

민지와 유사도가 가장 높은 사람 현우
현우의 노인과바다 평점이 가장 많이 적용될 것이다.
'''

def predict_ratings(user_name,book_name):
    neighbors_ratings = ratings[book_name].drop(index=user_name) #민지를 제외한 neighbors_ratings
    #neighbors_ratings

    neighbors_similarity = similarity_table[user_name].drop(index=user_name) #민지를 제외한 나머지 유사도
    neighbors_similarity

    #평점과 유사도를 각각 곱한다.
    # (민수평점 * 민수 유사도) + (지민평점 * 지민 유사도) + (지연평점 * 지연유사도)...
    nominator = (neighbors_ratings * neighbors_similarity).sum()

    #나를 제외한 유사도의 합
    denominator = neighbors_similarity.sum()

    #민지가 다른 사람과의 유사도, 평점을 봤을 때...스코어를 예측할 수 있다.
    score = nominator / denominator

    return score

In [55]:
predict_ratings("민지","노인과바다")

3.671361398092429

In [56]:
predict_ratings("민수","흥부전")

2.7543750620420546

### 모든 사용자와 상품에 대한 평점 검색

In [57]:
ratings.index, ratings.columns

(Index(['민수', '민지', '지민', '지연', '현우'], dtype='object', name='사람'),
 Index(['노인과바다', '백설공주', '신데렐라', '어린왕자', '콩쥐팥쥐', '흥부전'], dtype='object', name='책'))

In [76]:
rating_combinations = list(product(ratings.index, ratings.columns))
#rating_combinations

rating_list = []

for user_name, book_name in rating_combinations:
    score = predict_ratings(user_name, book_name)
    
    rating_predict = {
        "user":user_name,
        "book":book_name,
        "score":score
    }
    
    rating_list.append(rating_predict)
rating_list = pd.DataFrame(rating_list)
rating_list

rating_table = pd.pivot_table(rating_list,index="user",columns="book",values="score")
rating_table

rating_table2 = pd.pivot_table(rating_list,index="book",columns="user",values="score")
rating_table2

user,민수,민지,지민,지연,현우
book,,,,,
노인과바다,2.964046,3.671361,3.096369,2.585490,2.995806
백설공주,1.544704,1.298385,1.795672,2.392236,2.460760
신데렐라,2.334094,2.493374,3.109381,2.431305,2.710577
어린왕자,2.455289,2.685136,2.070415,2.283870,3.256756
콩쥐팥쥐,1.490775,2.522275,2.157241,1.790005,2.196302
흥부전,2.754375,1.850321,1.967154,1.983205,2.315152


### Case 1. 지금 민지에게 가장 추천해주고 싶은 책은?

In [66]:
def predict_book(user_name, k=1):
    predict_list = rating_table.loc[user_name].sort_values(ascending=False)
    predict_list = predict_list.head(k).index
    return predict_list

predict_book("민지")

Index(['노인과바다', '어린왕자'], dtype='object', name='book')

In [67]:
predict_book("현우")

Index(['어린왕자'], dtype='object', name='book')

### Case 2. 지금 백설공주 책에 가장 관심이 있을 것 같은 사용자는?

In [79]:
def predict_user(book_name,k=1):
    predict_list = rating_table2.loc[book_name].sort_values(ascending=False)
    predict_list = predict_list.head(1).index
    return predict_list
predict_user("백설공주")

Index(['현우'], dtype='object', name='user')

## Notation

$r_{ui}$를 사용자 $u$가 상품 $i$에 내린 ratings, $I_{uv}$ 를 사용자 $u$와 사용자 $v$가 모두 평가한 상품 집합, $U_{ij}$를 상품 $i$와 상품 $j$를 모두 평가한 사용자 집합이라고 표기하겠습니다.

![CF](C:/Users/Playdata/Desktop/git/TIL/Maching Learning/img/CF.png)

In [17]:
#피어슨 상관계수 구현

def get_pearson_similarity(u,v):
    mask = np.isfinite(u) & np.isfinite(v)
    u = u[mask]
    v = v[mask]
    
    #분자
    nvdot = ((u - u.mean()) * (v - v.mean())).sum()
    #분모
    norm3 = ((u - (u.sum())/len(u)) ** 2).sum()
    norm4 = ((v - (v.sum())/len(v)) ** 2).sum()
    
    score = nvdot / np.sqrt(norm3 * norm4)
    return score

In [18]:
## 민수, 민지에 대한 가로행 정보를 받아온다.
u = ratings.loc["민수"]
v = ratings.loc["민지"]
get_pearson_similarity(u,v)

0.9707253433941508

In [19]:
ratings

책,노인과바다,백설공주,신데렐라,어린왕자,콩쥐팥쥐,흥부전
사람,,,,,,
민수,3.0,4.0,4.0,3.0,4.0,NaN
민지,NaN,5.0,4.0,1.0,NaN,3.0
지민,4.0,1.0,NaN,5.0,2.0,3.0
지연,5.0,NaN,3.0,4.0,3.0,3.0
현우,3.0,NaN,2.0,NaN,1.0,2.0


In [20]:
ratings.index

Index(['민수', '민지', '지민', '지연', '현우'], dtype='object', name='사람')

### 모든 사람의 유사도를 검색

In [21]:
from itertools import product

In [22]:
def get_pearson_similarity_table(ratings):
    # ratings.index에서 2명씩 묶겠다...
    index_combinations=list(product(ratings.index, repeat=2))
    similarity_list = []

    for uname, vname in index_combinations:
        u, v = ratings.loc[uname], ratings.loc[vname]    
        score=get_pearson_similarity(u,v)

        similarity = { #json 형식
            'u':uname,
            'v':vname,
            'score':score        
        }    
        similarity_list.append(similarity)    
    similarity_list = pd.DataFrame(similarity_list)

    similarity_table = pd.pivot_table(similarity_list, index='u',columns='v', values='score')
    return similarity_table

In [23]:
similarity_table = get_pearson_similarity_table(ratings)
similarity_table

<ipython-input-17-c024d5380870>:14: RuntimeWarning: invalid value encountered in double_scalars
  score = nvdot / np.sqrt(norm3 * norm4)


v,민수,민지,지민,지연,현우
u,,,,,
민수,1.000000,0.970725,-0.948683,-0.904534,-0.866025
민지,0.970725,1.000000,-1.000000,-0.944911,NaN
지민,-0.948683,-1.000000,1.000000,0.674200,1.000000
지연,-0.904534,-0.944911,0.674200,1.000000,0.816497
현우,-0.866025,NaN,1.000000,0.816497,1.000000
